In [25]:
print('Importing packages...')
from datetime import datetime
import numpy as np
import pandas as pd
import xgboost
import re
import warnings
from warnings import filterwarnings
filterwarnings("ignore")
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

Importing packages...


In [14]:
print('Reading data...')
dfTest = pd.read_csv('test_indessa.csv')
dfTrain = pd.read_csv('train_indessa.csv')

dfTrain = dfTrain[['member_id', 'loan_amnt', 'funded_amnt', 'addr_state', 'funded_amnt_inv', 'sub_grade', 'term', 'desc', 'emp_length', 'int_rate', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc','total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'last_week_pay', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'loan_status']]
dfTest = dfTest[['member_id', 'loan_amnt', 'funded_amnt', 'addr_state', 'funded_amnt_inv', 'sub_grade', 'term', 'desc','emp_length', 'int_rate', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'last_week_pay', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim']]

Reading data...


In [18]:
dfTrain.describe

<bound method NDFrame.describe of         member_id  loan_amnt  funded_amnt addr_state  funded_amnt_inv  \
0        58189336      14350        14350         FL          14350.0   
1        70011223       4800         4800         MD           4800.0   
2        70255675      10000        10000         OH          10000.0   
3         1893936      15000        15000         VA          15000.0   
4         7652106      16000        16000         CA          16000.0   
...           ...        ...          ...        ...              ...   
532423   31296187      20000        20000         MI          20000.0   
532424   29403184      12000        12000         MI          12000.0   
532425    7357607      18725        18725         TN          18725.0   
532426   23182668      21000        21000         OH          21000.0   
532427   46122259      10000        10000         CA          10000.0   

       sub_grade  term                                               desc  \
0           

In [19]:
dfTrain.isnull().sum()

member_id                           0
loan_amnt                           0
funded_amnt                         0
addr_state                          0
funded_amnt_inv                     0
sub_grade                           0
term                                0
desc                           456829
emp_length                      26891
int_rate                            0
annual_inc                          3
dti                                 0
delinq_2yrs                        16
inq_last_6mths                     16
mths_since_last_delinq         272554
mths_since_last_record         450305
open_acc                           16
pub_rec                            16
revol_bal                           0
revol_util                        287
total_acc                          16
total_rec_int                       0
total_rec_late_fee                  0
recoveries                          0
collection_recovery_fee             0
collections_12_mths_ex_med         95
mths_since_l

In [20]:
dfTrain.head()

,member_id,loan_amnt,funded_amnt,addr_state,funded_amnt_inv,sub_grade,term,desc,emp_length,int_rate,...,recoveries,collection_recovery_fee,collections_12_mths_ex_med,mths_since_last_major_derog,last_week_pay,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,loan_status
0,58189336,14350,14350,FL,14350.0,E3,36,NaN,9 years,19.19,...,0.0,0.0,0.0,74.0,26,0.0,0.0,28699.0,30800.0,0
1,70011223,4800,4800,MD,4800.0,B4,36,NaN,< 1 year,10.99,...,0.0,0.0,0.0,NaN,9,0.0,0.0,9974.0,32900.0,0
2,70255675,10000,10000,OH,10000.0,A4,36,NaN,2 years,7.26,...,0.0,0.0,0.0,NaN,9,0.0,65.0,38295.0,34900.0,0
3,1893936,15000,15000,VA,15000.0,D5,36,> My goal is to obtain a loan to pay off my hi...,10+ years,19.72,...,0.0,0.0,0.0,NaN,135,0.0,0.0,55564.0,24700.0,0
4,7652106,16000,16000,CA,16000.0,B2,36,NaN,10+ years,10.64,...,0.0,0.0,0.0,NaN,96,0.0,0.0,47159.0,47033.0,0


In [21]:
dfTrain.describe

<bound method NDFrame.describe of         member_id  loan_amnt  funded_amnt addr_state  funded_amnt_inv  \
0        58189336      14350        14350         FL          14350.0   
1        70011223       4800         4800         MD           4800.0   
2        70255675      10000        10000         OH          10000.0   
3         1893936      15000        15000         VA          15000.0   
4         7652106      16000        16000         CA          16000.0   
...           ...        ...          ...        ...              ...   
532423   31296187      20000        20000         MI          20000.0   
532424   29403184      12000        12000         MI          12000.0   
532425    7357607      18725        18725         TN          18725.0   
532426   23182668      21000        21000         OH          21000.0   
532427   46122259      10000        10000         CA          10000.0   

       sub_grade  term                                               desc  \
0           

In [22]:
dfTest.isnull().sum()

member_id                           0
loan_amnt                           0
funded_amnt                         0
addr_state                          0
funded_amnt_inv                     0
sub_grade                           0
term                                0
desc                           304770
emp_length                      17934
int_rate                            0
annual_inc                          1
dti                                 0
delinq_2yrs                        13
inq_last_6mths                     13
mths_since_last_delinq         181758
mths_since_last_record         300021
open_acc                           13
pub_rec                            13
revol_bal                           0
revol_util                        215
total_acc                          13
total_rec_int                       0
total_rec_late_fee                  0
recoveries                          0
collection_recovery_fee             0
collections_12_mths_ex_med         50
mths_since_l

In [23]:
'''
Data transformation/cleanup
Strip off textual parts, represent values as numeric values
it makes sense. Convert the datatype to numeric.
'''
print('Transform: term...')
dfTrain['term'].replace(to_replace=' months', value='', regex=True, inplace=True)
dfTest['term'].replace(to_replace=' months', value='', regex=True, inplace=True)
dfTrain['term'] = pd.to_numeric(dfTrain['term'], errors='coerce')
dfTest['term'] = pd.to_numeric(dfTest['term'], errors='coerce')

print('Transform: emp_length...')
dfTrain['emp_length'].replace('n/a', '0', inplace=True)
dfTrain['emp_length'].replace(to_replace='\+ years', value='', regex=True, inplace=True)
dfTrain['emp_length'].replace(to_replace=' years', value='', regex=True, inplace=True)
dfTrain['emp_length'].replace(to_replace='< 1 year', value='0', regex=True, inplace=True)
dfTrain['emp_length'].replace(to_replace=' year', value='', regex=True, inplace=True)
dfTest['emp_length'].replace('n/a', '0', inplace=True)
dfTest['emp_length'].replace(to_replace='\+ years', value='', regex=True, inplace=True)
dfTest['emp_length'].replace(to_replace=' years', value='', regex=True, inplace=True)
dfTest['emp_length'].replace(to_replace='< 1 year', value='0', regex=True, inplace=True)
dfTest['emp_length'].replace(to_replace=' year', value='', regex=True, inplace=True)
dfTrain['emp_length'] = pd.to_numeric(dfTrain['emp_length'], errors='coerce')
dfTest['emp_length'] = pd.to_numeric(dfTest['emp_length'], errors='coerce')

print('Transform: last_week_pay...')
dfTrain['last_week_pay'].replace(to_replace='th week', value='', regex=True, inplace=True)
dfTest['last_week_pay'].replace(to_replace='th week', value='', regex=True, inplace=True)
dfTrain['last_week_pay'].replace(to_replace='NA', value='', regex=True, inplace=True)
dfTest['last_week_pay'].replace(to_replace='NA', value='', regex=True, inplace=True)
dfTrain['last_week_pay'] = pd.to_numeric(dfTrain['last_week_pay'], errors='coerce')
dfTest['last_week_pay'] = pd.to_numeric(dfTest['last_week_pay'], errors='coerce')

print('Transform: sub_grade...')
dfTrain['sub_grade'].replace(to_replace='A', value='0', regex=True, inplace=True)
dfTrain['sub_grade'].replace(to_replace='B', value='1', regex=True, inplace=True)
dfTrain['sub_grade'].replace(to_replace='C', value='2', regex=True, inplace=True)
dfTrain['sub_grade'].replace(to_replace='D', value='3', regex=True, inplace=True)
dfTrain['sub_grade'].replace(to_replace='E', value='4', regex=True, inplace=True)
dfTrain['sub_grade'].replace(to_replace='F', value='5', regex=True, inplace=True)
dfTrain['sub_grade'].replace(to_replace='G', value='6', regex=True, inplace=True)
dfTest['sub_grade'].replace(to_replace='A', value='0', regex=True, inplace=True)
dfTest['sub_grade'].replace(to_replace='B', value='1', regex=True, inplace=True)
dfTest['sub_grade'].replace(to_replace='C', value='2', regex=True, inplace=True)
dfTest['sub_grade'].replace(to_replace='D', value='3', regex=True, inplace=True)
dfTest['sub_grade'].replace(to_replace='E', value='4', regex=True, inplace=True)
dfTest['sub_grade'].replace(to_replace='F', value='5', regex=True, inplace=True)
dfTest['sub_grade'].replace(to_replace='G', value='6', regex=True, inplace=True)
dfTrain['sub_grade'] = pd.to_numeric(dfTrain['sub_grade'], errors='coerce')
dfTest['sub_grade'] = pd.to_numeric(dfTest['sub_grade'], errors='coerce')

print('Transform done.')

Transform: term...
Transform: emp_length...
Transform: last_week_pay...
Transform: sub_grade...
Transform done.


In [26]:
# Separating the member_id column of test dataframe to help create a csv after predictions
test_member_id = pd.DataFrame(dfTest['member_id'])


# Creating target variable pandas series from train dataframe, this will be used by cross validation to calculate
# the accuracy of the model
train_target = pd.DataFrame(dfTrain['loan_status'])


# It's good to create a copy of train and test dataframes. this way we can play around different features as we tune the
# performance of the classifier with important features
selected_cols = ['member_id', 'emp_length', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'sub_grade', 'int_rate', 'annual_inc', 'dti', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'revol_bal', 'revol_util', 'total_acc', 'total_rec_int', 'total_rec_late_fee', 'mths_since_last_major_derog', 'last_week_pay', 'tot_cur_bal', 'total_rev_hi_lim', 'tot_coll_amt', 'recoveries', 'collection_recovery_fee', 'term', 'acc_now_delinq', 'collections_12_mths_ex_med']
finalTrain = dfTrain[selected_cols]
finalTest = dfTest[selected_cols]

# How big the loan a person has taken with respect to his earnings, annual income to loan amount ratio
finalTrain['loan_to_income'] = finalTrain['annual_inc']/finalTrain['funded_amnt_inv']
finalTest['loan_to_income'] = finalTest['annual_inc']/finalTest['funded_amnt_inv']


# All these attributes indicate that the repayment was not all hunky-dory. All the amounts caclulated are ratios 
# like, recovery to the loan amount. This column gives a magnitude of how much the repayment has gone off course 
# in terms of ratios.
finalTrain['bad_state'] = finalTrain['acc_now_delinq'] + (finalTrain['total_rec_late_fee']/finalTrain['funded_amnt_inv']) + (finalTrain['recoveries']/finalTrain['funded_amnt_inv']) + (finalTrain['collection_recovery_fee']/finalTrain['funded_amnt_inv']) + (finalTrain['collections_12_mths_ex_med']/finalTrain['funded_amnt_inv'])
finalTest['bad_state'] = finalTest['acc_now_delinq'] + (finalTest['total_rec_late_fee']/finalTest['funded_amnt_inv']) + (finalTest['recoveries']/finalTest['funded_amnt_inv']) + (finalTest['collection_recovery_fee']/finalTest['funded_amnt_inv']) + (finalTrain['collections_12_mths_ex_med']/finalTest['funded_amnt_inv'])

# For the sake of this model, I have used just a boolean flag if things had gone bad, with this case I didn't see
# a benifit of including above computations
finalTrain.loc[finalTrain['bad_state'] > 0, 'bad_state'] = 1
finalTest.loc[finalTest['bad_state'] > 0, 'bad_state'] = 1


# Total number of available/unused 'credit lines'
finalTrain['avl_lines'] = finalTrain['total_acc'] - finalTrain['open_acc']
finalTest['avl_lines'] = finalTest['total_acc'] - finalTest['open_acc']


# Interest paid so far
finalTrain['int_paid'] = finalTrain['total_rec_int'] + finalTrain['total_rec_late_fee']
finalTest['int_paid'] = finalTest['total_rec_int'] + finalTest['total_rec_late_fee']


# Calculating EMIs paid (in terms of percent)
finalTrain['emi_paid_progress_perc'] = ((finalTrain['last_week_pay']/(finalTrain['term']/12*52+1))*100)
finalTest['emi_paid_progress_perc'] = ((finalTest['last_week_pay']/(finalTest['term']/12*52+1))*100)


# Calculating total repayments received so far, in terms of EMI or recoveries after charge off
finalTrain['total_repayment_progress'] = ((finalTrain['last_week_pay']/(finalTrain['term']/12*52+1))*100) + ((finalTrain['recoveries']/finalTrain['funded_amnt_inv']) * 100)
finalTest['total_repayment_progress'] = ((finalTest['last_week_pay']/(finalTest['term']/12*52+1))*100) + ((finalTest['recoveries']/finalTest['funded_amnt_inv']) * 100)


In [27]:
#Null values in training dataset

null= finalTrain.isnull().sum().sort_values(ascending=False)
total =finalTrain.shape[0]
percent_missing= (finalTrain.isnull().sum()/total).sort_values(ascending=False)

missing_data= pd.concat([null, percent_missing], axis=1, keys=['Total missing', 'Percent missing'])

missing_data.reset_index(inplace=True)
missing_data= missing_data.rename(columns= { "index": " column name"})
 
print ("Null Values in each column:\n", missing_data.sort_values(by ='Total missing', ascending = False))

Null Values in each column:
                     column name  Total missing  Percent missing
0        mths_since_last_record         450305         0.845758
1   mths_since_last_major_derog         399448         0.750239
2        mths_since_last_delinq         272554         0.511908
3                  tot_coll_amt          42004         0.078891
4              total_rev_hi_lim          42004         0.078891
5                   tot_cur_bal          42004         0.078891
6                    emp_length          26891         0.050506
7                    revol_util            287         0.000539
8                     bad_state            228         0.000428
9      total_repayment_progress            113         0.000212
10   collections_12_mths_ex_med             95         0.000178
11               acc_now_delinq             16         0.000030
12                    total_acc             16         0.000030
13                    avl_lines             16         0.000030
14         

In [28]:

null= finalTest.isnull().sum().sort_values(ascending=False)
total =finalTest.shape[0]
percent_missing= (finalTest.isnull().sum()/total).sort_values(ascending=False)

missing_data= pd.concat([null, percent_missing], axis=1, keys=['Total missing', 'Percent missing'])

missing_data.reset_index(inplace=True)
missing_data= missing_data.rename(columns= { "index": " column name"})
 
print ("Null Values in each column:\n", missing_data.sort_values(by ='Total missing', ascending = False))

Null Values in each column:
                     column name  Total missing  Percent missing
0        mths_since_last_record         300021         0.845246
1   mths_since_last_major_derog         266228         0.750042
2        mths_since_last_delinq         181758         0.512065
3                  tot_coll_amt          28272         0.079650
4              total_rev_hi_lim          28272         0.079650
5                   tot_cur_bal          28272         0.079650
6                    emp_length          17934         0.050525
7      total_repayment_progress           7129         0.020084
8                 last_week_pay           7045         0.019848
9        emi_paid_progress_perc           7045         0.019848
10                   revol_util            215         0.000606
11                    bad_state            170         0.000479
12   collections_12_mths_ex_med             50         0.000141
15                    avl_lines             13         0.000037
16         

In [29]:
def fill_nulls(value):
    cols_fill = ['mths_since_last_record','mths_since_last_major_derog',
                 'mths_since_last_delinq','total_rev_hi_lim','tot_cur_bal',
                 'tot_coll_amt','emp_length','revol_util','collections_12_mths_ex_med',
                 'open_acc','total_acc','acc_now_delinq','avl_lines','loan_to_income',
                 'annual_inc','bad_state','total_repayment_progress']
    
    if value == -9999:
        for col in cols_fill:
            finalTest.loc[finalTest[col].isnull(), col] = -9999
    else : 
        for col in cols_fill:
            finalTest.loc[finalTest[col].isnull(), col] = finalTrain[col].median()

In [30]:
fill_nulls(0)

In [31]:
def fill_nulls(value):
    cols_fill = ['mths_since_last_record','mths_since_last_major_derog',
                 'mths_since_last_delinq','total_rev_hi_lim','tot_cur_bal',
                 'tot_coll_amt','emp_length','revol_util','collections_12_mths_ex_med',
                 'open_acc','total_acc','acc_now_delinq','avl_lines','loan_to_income',
                 'annual_inc','bad_state','total_repayment_progress']
    if value == -9999:
        for col in cols_fill:
            finalTrain.loc[finalTrain[col].isnull(), col] = -9999
    else : 
        for col in cols_fill:
            finalTrain.loc[finalTrain[col].isnull(), col] = finalTrain[col].median()

In [32]:
fill_nulls(0)

In [33]:
finalTrain = finalTrain.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

In [34]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_trainScaled = scaler.fit_transform(finalTrain)

In [35]:
# Split train and cross validation sets
X_train, X_test, y_train, y_test = train_test_split(np.array(finalTrain), np.array(train_target), test_size=0.30)
eval_set=[(X_test, y_test)]

In [36]:
print('Initializing xgboost.sklearn.XGBClassifier....')


clf = xgboost.sklearn.XGBClassifier(
    objective="binary:logistic", 
    learning_rate=0.09, 
    seed=9616, 
    max_depth=30, 
    gamma=10, 
    n_estimators=500)

Initializing xgboost.sklearn.XGBClassifier....


In [37]:
print("Training Started")
clf.fit(X_train, y_train, early_stopping_rounds=20, eval_metric="auc", eval_set=eval_set, verbose=True)

y_pred = clf.predict(X_test)
accuracy = accuracy_score(np.array(y_test).flatten(), y_pred)

Training Started
[0]	validation_0-auc:0.96745
[1]	validation_0-auc:0.96998
[2]	validation_0-auc:0.97090
[3]	validation_0-auc:0.97112
[4]	validation_0-auc:0.97164
[5]	validation_0-auc:0.97176
[6]	validation_0-auc:0.97188
[7]	validation_0-auc:0.97224
[8]	validation_0-auc:0.97243
[9]	validation_0-auc:0.97246
[10]	validation_0-auc:0.97268
[11]	validation_0-auc:0.97282
[12]	validation_0-auc:0.97298
[13]	validation_0-auc:0.97308
[14]	validation_0-auc:0.97328
[15]	validation_0-auc:0.97337
[16]	validation_0-auc:0.97344
[17]	validation_0-auc:0.97348
[18]	validation_0-auc:0.97358
[19]	validation_0-auc:0.97360
[20]	validation_0-auc:0.97362
[21]	validation_0-auc:0.97371
[22]	validation_0-auc:0.97373
[23]	validation_0-auc:0.97378
[24]	validation_0-auc:0.97381
[25]	validation_0-auc:0.97387
[26]	validation_0-auc:0.97395
[27]	validation_0-auc:0.97403
[28]	validation_0-auc:0.97408
[29]	validation_0-auc:0.97413
[30]	validation_0-auc:0.97419
[31]	validation_0-auc:0.97428
[32]	validation_0-auc:0.97435
[33

In [38]:
submission_file_name = "Loan_Defaulter_submission"
print("Accuracy: %.10f%%" % (accuracy * 100.0))
submission_file_name = submission_file_name + ("_Accuracy_%.6f" % (accuracy * 100)) + '_'

accuracy_per_roc_auc = roc_auc_score(np.array(y_test).flatten(), y_pred)
print("ROC-AUC: %.10f%%" % (accuracy_per_roc_auc * 100))
submission_file_name = submission_file_name + ("_ROC-AUC_%.6f" % (accuracy_per_roc_auc * 100))

Accuracy: 94.0874856789%
ROC-AUC: 92.3986358528%


In [39]:
final_pred = pd.DataFrame(clf.predict_proba(np.array(finalTest)))
dfSub = pd.concat([test_member_id, final_pred.iloc[:, 1:2]], axis=1)
dfSub.rename(columns={1:'loan_status'}, inplace=True)
dfSub.to_csv((('%s.csv') % (submission_file_name)), index=False)

NameError: name 'knn' is not defined